In [1]:
import numpy as np
import mltools as ml
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.feature_selection import SelectPercentile
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.ensemble import ExtraTreesClassifier

def toKaggle(YeHat):
    fh = open("Y_submit.txt","w")
    fh.write("ID,Predicted\n")
    for i,yi in enumerate(YeHat.ravel()[:7422]):
        fh.write(f"{i},{yi}\n")
    fh.close()

X = np.genfromtxt('X_train.txt', delimiter=',')
Y = np.genfromtxt('Y_train.txt', delimiter=',')
Xte = np.genfromtxt('X_test.txt',delimiter =',')
X,Y = ml.shuffleData(X,Y)








In [2]:

#Binary data was tested and had poor scores w.r.t importance of each feature 
Xt = preprocessing.normalize(X,axis = 0)[:,:69] #Normalize non-binary data

'''
This was used to determine the importance of each feature 
model = ExtraTreesClassifier(n_estimators = 20)
model.fit(Xt,Y)
ft = list()
for i,v in enumerate(model.feature_importances_):
    ft.append((i,v))
ft = sorted(ft,key = lambda x:x[1],reverse = True)
for f in ft:
    print(f)
'''





sf = SelectPercentile(f_classif, percentile=7)
fit = sf.fit(Xt,Y)
features = fit.transform(Xt)
Xtr,Xva,Ytr,Yva = ml.splitData(features,Y, 0.50)


n = KNeighborsClassifier(n_neighbors = 33,metric = "manhattan")
n.fit(Xtr,Ytr)




y_scores = n.predict_proba(Xtr)
fpr, tpr, threshold = roc_curve(Ytr,y_scores[:, 1])
roc_auc = auc(fpr, tpr)
print("Training AUC: ",roc_auc)
y_scores = n.predict_proba(Xva)
fpr, tpr, threshold = roc_curve(Yva, y_scores[:, 1])
roc_auc = auc(fpr, tpr)
print("Validation AUC: ",roc_auc)


featuresT = fit.transform(Xte[:,:69])
pre = n.predict_proba(featuresT)[:,1]
toKaggle(pre)





Training AUC:  0.7323492629276348
Validation AUC:  0.6867294080878167
